# Introduction to geospatial machine learning
Here, we are going to introduce the basic concepts of geospatial operations including those of relations and spatial joins.

Let's import all the necessary libraries and setup our dataframe :

In [7]:
# Import the necessary libraries
import geopandas as gpd
import shapely.geometry
import matplotlib.pyplot as plt

# Initialize our geodataframe (gdf)
path_to_data = gpd.datasets.get_path("nybb")
gdf = gpd.read_file(path_to_data)

# Set borough name as index
gdf = gdf.set_index("BoroName")

# Get buffered region and centroid points
gdf["buffered"] = gdf.buffer(10000)
gdf['centroid'] = gdf.centroid

### Geospatial relations
Geometries, such as the polygons that define the individual borough or lines that represent the boundaries between the regions or points to indicate the centroid positions, can be related to one another.

#### A. Intersects
Suppose we ask the following question: does the buffer region intersect with Brooklyn borough in New York?

First we get the polygon for Brooklyn

In [8]:
brooklyn = gdf.loc["Brooklyn", "geometry"]

We can then chec if the buffered region intersects it :

In [9]:
gdf["buffered"].intersects(brooklyn)

BoroName
Staten Island     True
Queens            True
Brooklyn          True
Manhattan         True
Bronx            False
dtype: bool

It seems that Bronx is very far away from the Brooklyn that a buffer area of 10,000 feet beyond the boundary does not intersect Brooklyn at all.

#### C. Within
Another question could similarly arise: are the centroids contained in their respective boroughs (ie. unique centroids)? This could be performed by within relation that specifies whether a collection of points are contained in the said polygons.

In [10]:
gdf["centroid"].within(brooklyn)

BoroName
Staten Island    False
Queens           False
Brooklyn          True
Manhattan        False
Bronx            False
dtype: bool

To be expected, the centroid of Brooklyn is located within Brooklyn polygon and not within the other regions.

### Spatial Joins
Another important geospatial operations that we could perform with geopandas is spatial joins. As the name suggests, it allows two distinct geometries to be connected based on certain geometric criteria.

Let us suppose we have a few bus stations within the area, and we wanted to identify the borough in which each of the station lies.

In [12]:
#generate points
b = [int(x) for x in gdf.total_bounds]
N = 6

pointdf = gpd.GeoDataFrame([
    {'geometry': shapely.geometry.Point(x, y), 'value1':x+y, 'value2':x-y}
    for x, y in zip(range(b[0], b[2], int((b[2] - b[0]) / N)),
                    range(b[1], b[3], int((b[3] - b[1]) / N)))
    ])

#Make sure they are using the same projection reference
pointdf.crs = gdf.crs

The first solution that comes to mind would be to perform a within operation as before, and then for each borough, the True index value will be assigned to that point (eg. if bus station 1 is within Brooklyn, assigns Brooklyn to the point item).

The spatial join can then be performed as follows:

In [14]:
from geopandas.tools import sjoin

# Joining the NYC borough INTO the pointdf
sjoin(pointdf, gdf, how="inner", op="intersects")

#We choose inner option to avoid having a lot of NaNs

,geometry,value1,value2,index_right,BoroCode,Shape_Leng,Shape_Area,buffered,centroid
1,POINT (938876.000 145574.000),1084450,793302,Staten Island,5,330470.010332,1.623820e+09,"POLYGON ((903234.894 123347.784, 903178.057 12...",POINT (941639.450 150931.991)
